In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
files = os.listdir('ham')
data = []
for file in files:
    temp = pd.read_table('ham/'+file,sep = '/n',header = None)
    text = ''
    for x in temp[0]:
        text = text+x
    data.append(text)

C:\Users\施梦娴\AppData\Local\Temp\ipykernel_8764\488856658.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  temp = pd.read_table('ham/'+file,sep = '/n',header = None)
C:\Users\施梦娴\AppData\Local\Temp\ipykernel_8764\488856658.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  temp = pd.read_table('ham/'+file,sep = '/n',header = None)
C:\Users\施梦娴\AppData\Local\Temp\ipykernel_8764\488856658.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by 

In [3]:
data = pd.DataFrame(data)
data.columns = ['text']
data['spam'] = 0

In [4]:
files = os.listdir('spam')
data1 = []
for file in files:
    try:
        temp = pd.read_table('spam/'+file,sep = '/n',header = None)
    except: 
        continue
    text = ''
    for x in temp[0]:
        text = text+x
    data1.append(text)
data1 = pd.DataFrame(data1)
data1.columns = ['text']
data1['spam'] = 1

C:\Users\施梦娴\AppData\Local\Temp\ipykernel_8764\2001353183.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  temp = pd.read_table('spam/'+file,sep = '/n',header = None)
C:\Users\施梦娴\AppData\Local\Temp\ipykernel_8764\2001353183.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  temp = pd.read_table('spam/'+file,sep = '/n',header = None)
C:\Users\施梦娴\AppData\Local\Temp\ipykernel_8764\2001353183.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warnin

In [5]:
print('有效读取',data1.shape[0]/len(files))

有效读取 0.99


In [6]:
data = pd.concat([data,data1])

In [8]:
data['spam'].value_counts()

0    3672
1    1485
Name: spam, dtype: int64

In [9]:
# 分层抽样
from sklearn.model_selection import StratifiedShuffleSplit
splt = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=1)
for train_idx, test_idx in splt.split(data, data['spam']): # 按照后者分层抽样
    train = data.iloc[train_idx]
    test = data.iloc[test_idx]
# 查看分布
print(train['spam'].value_counts()/len(train))
print(test['spam'].value_counts()/len(test))

0    0.712
1    0.288
Name: spam, dtype: float64
0    0.712209
1    0.287791
Name: spam, dtype: float64


In [151]:
train[train['spam']==1]['text'].str.strip().str.len().mean()

1159.5681818181818

In [152]:
train[train['spam']==0]['text'].str.strip().str.len().mean()

941.9911474293497

In [102]:
x_train = train['text'].values
y_train = train['spam']
x_test = test['text'].values
y_test = test['spam']

In [74]:
from sklearn.feature_extraction.text import CountVectorizer

In [105]:
vect = CountVectorizer().fit(x_train)
x_train_vectorized = vect.transform(x_train)

In [104]:
pd.DataFrame(vect.vocabulary_,index = [0]).T.sort_values(by = 0,ascending=False)[:20]

,0
zzso,63598
zzo,63597
zzn,63596
zzezrjok,63595
zyyqywp,63594
zyqtaqlt,63593
zynve,63592
zynsdirnh,63591
zyl,63590
zykfe,63589


In [114]:
# 朴素贝叶斯
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

model = MultinomialNB(alpha=0.1).fit(x_train_vectorized, y_train)
predictions = model.predict(vect.transform(x_test))


def analysis(y_test,predictions):
    aucscore = roc_auc_score(y_test, predictions) 
    print('auc: ',aucscore)
    tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
    print(pd.DataFrame(confusion_matrix(y_test, predictions),columns=['Predicted Spam', "Predicted Ham"], index=['Actual Spam', 'Actual Ham']))
    print(f'\nTrue Positives: {tp}')
    print(f'False Positives: {fp}')
    print(f'True Negatives: {tn}')
    print(f'False Negatives: {fn}')

    print(f'True Positive Rate: { (tp / (tp + fn))}')
    print(f'Specificity: { (tn / (tn + fp))}')
    print(f'False Positive Rate: { (fp / (fp + tn))}')
    return

analysis(y_test,predictions)

auc:  0.9672301243729815
             Predicted Spam  Predicted Ham
Actual Spam             719             16
Actual Ham               13            284

True Positives: 284
False Positives: 16
True Negatives: 719
False Negatives: 13
True Positive Rate: 0.9562289562289562
Specificity: 0.9782312925170068
False Positive Rate: 0.021768707482993196


In [115]:
#逻辑回归
from sklearn.linear_model import LogisticRegression
logistic_regression = LogisticRegression().fit(x_train_vectorized, y_train)
predictions_lr = logistic_regression.predict(vect.transform(x_test))
analysis(y_test, predictions_lr) 

auc:  0.9668384525527383
             Predicted Spam  Predicted Ham
Actual Spam             711             24
Actual Ham               10            287

True Positives: 287
False Positives: 24
True Negatives: 711
False Negatives: 10
True Positive Rate: 0.9663299663299664
Specificity: 0.9673469387755103
False Positive Rate: 0.0326530612244898


c:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [116]:
# 支持向量机
from sklearn import svm
svm_clf = svm.SVC(gamma='scale').fit(x_train_vectorized, y_train)
predictions_svm = svm_clf.predict(vect.transform(x_test))
analysis(y_test,predictions_svm)

auc:  0.9495774067202638
             Predicted Spam  Predicted Ham
Actual Spam             698             37
Actual Ham               15            282

True Positives: 282
False Positives: 37
True Negatives: 698
False Negatives: 15
True Positive Rate: 0.9494949494949495
Specificity: 0.9496598639455782
False Positive Rate: 0.050340136054421766


In [117]:
#knn
from sklearn.neighbors import KNeighborsClassifier
k_neighbor = KNeighborsClassifier()
k_neighbor.fit(x_train_vectorized, y_train)
predictions_knn = k_neighbor.predict(vect.transform(x_test))
analysis(y_test,predictions_knn)

auc:  0.8492407063835635
             Predicted Spam  Predicted Ham
Actual Spam             600            135
Actual Ham               35            262

True Positives: 262
False Positives: 135
True Negatives: 600
False Negatives: 35
True Positive Rate: 0.8821548821548821
Specificity: 0.8163265306122449
False Positive Rate: 0.1836734693877551


c:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [119]:
#决策树
from sklearn.tree import DecisionTreeClassifier
decision_tree = DecisionTreeClassifier()
decision_tree.fit(x_train_vectorized, y_train)
predictions_dt = decision_tree.predict(vect.transform(x_test))
analysis(y_test,predictions_dt)

auc:  0.9232529375386519
             Predicted Spam  Predicted Ham
Actual Spam             689             46
Actual Ham               27            270

True Positives: 270
False Positives: 46
True Negatives: 689
False Negatives: 27
True Positive Rate: 0.9090909090909091
Specificity: 0.9374149659863945
False Positive Rate: 0.06258503401360545


In [120]:
# 随机森林
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier()
random_forest.fit(x_train_vectorized, y_train)
predictions_rf = random_forest.predict(vect.transform(x_test))
analysis(y_test,predictions_rf)

auc:  0.9463478320621178
             Predicted Spam  Predicted Ham
Actual Spam             718             17
Actual Ham               25            272

True Positives: 272
False Positives: 17
True Negatives: 718
False Negatives: 25
True Positive Rate: 0.9158249158249159
Specificity: 0.9768707482993197
False Positive Rate: 0.02312925170068027


In [121]:
#GBDT
from sklearn.ensemble import GradientBoostingClassifier
gdbt = GradientBoostingClassifier()
gdbt.fit(x_train_vectorized, y_train)
predictions_gdbt = gdbt.predict(vect.transform(x_test))
analysis(y_test,predictions_gdbt)

auc:  0.9371126228269087
             Predicted Spam  Predicted Ham
Actual Spam             697             38
Actual Ham               22            275

True Positives: 275
False Positives: 38
True Negatives: 697
False Negatives: 22
True Positive Rate: 0.9259259259259259
Specificity: 0.9482993197278912
False Positive Rate: 0.05170068027210884


In [124]:
#神经网络
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(solver='lbfgs', activation='logistic')
mlp.fit(x_train_vectorized, y_train)
predictions_nn = mlp.predict(vect.transform(x_test))
analysis(y_test,predictions_nn)

auc:  0.9618223046794475
             Predicted Spam  Predicted Ham
Actual Spam             716             19
Actual Ham               15            282

True Positives: 282
False Positives: 19
True Negatives: 716
False Negatives: 15
True Positive Rate: 0.9494949494949495
Specificity: 0.9741496598639455
False Positive Rate: 0.02585034013605442
